# Week 5, Back-propagation

## Further reading

* Elements of Statistical Learning: Chapter 11, Neural Networks

## Code

### Feed-forward basics

To get started, we'll need to import `numpy` to deal with all the matrices involved.

In [ ]:
import numpy

The feed-forward of a nmeural network runs input through the network's weights and actication functions to produce an output. The first "layer" of the NN is called the input layer, and it's merely a copy of the input data. The first active layer is the first of the NN's hidden layers. Here is what it does:

$$z_1 = B_1 + x W_1$$
$$a_1 = \sigma(z_1)$$

Let's look at that more closely. $B_1$ (biases) and $W_1$ (weights) are the "neurons" of the first layer. The convention is that their dimensions are $in \times out$: $in$ is the number of input variables and $out$ is the number of neurons emitting output signals. The number of columns in the biases and weights are the neurons that individually perform some work on the incoming data.

The crucial part of the neural network is that $x$ is matrix multiplied by $W_1$, which is $1 by in \times in by out$ giving $1 by out$. The properties of matrix multiplication being what they are, each neuron "works" on the whole data independently and outputs a value separately.

The data is attended by multiple neurons, each able to perform its own processing of the data.

The feed-forward is then a great mixing of data among layers of neurons, finally creating a network output. This "densely connected" NN has each neuron working separately from its neighbors but seeing the whole of the previous layer's data. These plentiful connections mean that the NN can model much more complicated functions.

With all that in mind, this is the feed-forward:

$$z_1 = B_1 + x W_1$$
$$a_1 = \sigma(z_1)$$
$$z_2 = B_2 + a_1 W_2$$
$$a_2 = \sigma(z_2)$$
$$z_{output} = B_{output} + a_2 W_{output}$$
$$a_{output} = \sigma(z_{output})$$